In [1]:
import numpy as np
from Functions.combine_matrices import combine_matrices
from Functions.encode import encode

Wavelets Transform

In [2]:
cA2 = [[14, 28],
       [22, 24]]
cH2 = [[7, 4],
       [9, 10]]
cV2 = [[-10, 8],
       [-9, 11]]
cD2 = [[6, 8],
       [7, 4]]

cA1 = combine_matrices(cA2, cH2, cV2, cD2)
cH1 = [[4, 6, 8, 6],
       [-2, -3, -4, -2],
       [-4, -5, -6, -5],
       [3, 4, 6, 5]]
cV1 = [[10, -5, 4, 7],
       [9, -7, 6, 8],
       [9, -6, 5, 10],
       [8, -5, 8, 9]]
cD1 = [[2, 4, 2, 6],
       [1, 3, -2, -4],
       [3, -2, -6, -8],
       [2, 3, -1, -2]]

f = combine_matrices(cA1, cH1, cV1, cD1)

display(f)

array([[ 14,  28,   7,   4,   4,   6,   8,   6],
       [ 22,  24,   9,  10,  -2,  -3,  -4,  -2],
       [-10,   8,   6,   8,  -4,  -5,  -6,  -5],
       [ -9,  11,   7,   4,   3,   4,   6,   5],
       [ 10,  -5,   4,   7,   2,   4,   2,   6],
       [  9,  -7,   6,   8,   1,   3,  -2,  -4],
       [  9,  -6,   5,  10,   3,  -2,  -6,  -8],
       [  8,  -5,   8,   9,   2,   3,  -1,  -2]])

Scan order

In [3]:
cA2 = [[2, 4],
       [1, 3]]
cH2 = [[8 ,7],
       [5, 6]]
cV2 = [[9, 12],
       [10, 11]]
cD2 = [[14, 16],
       [13, 15]]
cA1 = combine_matrices(cA2, cH2, cV2, cD2)
cH1 = [[32, 31, 30, 29],
      [25, 26, 27, 28],
      [24, 23, 22, 21],
      [17, 18, 19, 20]]

cV1 = [[36, 37, 44, 45],
      [35, 38, 43, 46],
      [34, 39, 42, 47],
      [33, 40, 41, 48]]

cD1 = [[55, 61, 62, 64],
      [54, 56, 60, 63],
      [50, 53, 57, 59],
      [49, 51, 52, 58]]

scan_order = combine_matrices(cA1, cH1, cV1, cD1)

display(scan_order)

array([[ 2,  4,  8,  7, 32, 31, 30, 29],
       [ 1,  3,  5,  6, 25, 26, 27, 28],
       [ 9, 12, 14, 16, 24, 23, 22, 21],
       [10, 11, 13, 15, 17, 18, 19, 20],
       [36, 37, 44, 45, 55, 61, 62, 64],
       [35, 38, 43, 46, 54, 56, 60, 63],
       [34, 39, 42, 47, 50, 53, 57, 59],
       [33, 40, 41, 48, 49, 51, 52, 58]])

Преобразуем матрицу в последовательность координат

In [4]:
_, indx_array = np.unique(scan_order, return_index=True)
rows, cols = np.unravel_index(indx_array, f.shape)
indx_order = []
for r, c in zip(rows,cols):
    indx_order.append((r,c))

Проверка

In [5]:
for idx in indx_order:   
    print(f"{idx}: {scan_order[idx]}")

(1, 0): 1
(0, 0): 2
(1, 1): 3
(0, 1): 4
(1, 2): 5
(1, 3): 6
(0, 3): 7
(0, 2): 8
(2, 0): 9
(3, 0): 10
(3, 1): 11
(2, 1): 12
(3, 2): 13
(2, 2): 14
(3, 3): 15
(2, 3): 16
(3, 4): 17
(3, 5): 18
(3, 6): 19
(3, 7): 20
(2, 7): 21
(2, 6): 22
(2, 5): 23
(2, 4): 24
(1, 4): 25
(1, 5): 26
(1, 6): 27
(1, 7): 28
(0, 7): 29
(0, 6): 30
(0, 5): 31
(0, 4): 32
(7, 0): 33
(6, 0): 34
(5, 0): 35
(4, 0): 36
(4, 1): 37
(5, 1): 38
(6, 1): 39
(7, 1): 40
(7, 2): 41
(6, 2): 42
(5, 2): 43
(4, 2): 44
(4, 3): 45
(5, 3): 46
(6, 3): 47
(7, 3): 48
(7, 4): 49
(6, 4): 50
(7, 5): 51
(7, 6): 52
(6, 5): 53
(5, 4): 54
(4, 4): 55
(5, 5): 56
(6, 6): 57
(7, 7): 58
(6, 7): 59
(5, 6): 60
(4, 5): 61
(4, 6): 62
(5, 7): 63
(4, 7): 64


# Bit-plane Encoding Procedure

In [6]:
threshold = [16, 8, 4, 2, 1]
K = len(threshold)
st_arr = np.empty(K, dtype='object')
mse_err = []
for k in range(K):
    st_arr[k] = f.copy()
    for idx in indx_order:
        # Stage 1: Significance Pass only.
        if k == 0:
            if np.abs(st_arr[k][idx]) >= threshold[k]:
                st_arr[k][idx] = np.sign(st_arr[k][idx])*threshold[k]
            else:
                st_arr[k][idx] = 0
        # Stage >1: Both Significance Pass and Refinement Pass.
        else:
            if np.abs(st_arr[k][idx]) >= (np.abs(st_arr[k-1][idx]) + threshold[k]):
                st_arr[k][idx] = st_arr[k-1][idx] + np.sign(st_arr[k][idx])*threshold[k]
            else:
                st_arr[k][idx] = st_arr[k-1][idx]
    mse_err.append(np.square(f - st_arr[k]).mean())
for k in range(K):
    display(st_arr[k])
print('MSE:', np.round(mse_err,3))

array([[ 0, 16,  0,  0,  0,  0,  0,  0],
       [16, 16,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

array([[ 8, 24,  0,  0,  0,  0,  8,  0],
       [16, 24,  8,  8,  0,  0,  0,  0],
       [-8,  8,  0,  8,  0,  0,  0,  0],
       [-8,  8,  0,  0,  0,  0,  0,  0],
       [ 8,  0,  0,  0,  0,  0,  0,  0],
       [ 8,  0,  0,  8,  0,  0,  0,  0],
       [ 8,  0,  0,  8,  0,  0,  0, -8],
       [ 8,  0,  8,  8,  0,  0,  0,  0]])

array([[12, 28,  4,  4,  4,  4,  8,  4],
       [20, 24,  8,  8,  0,  0, -4,  0],
       [-8,  8,  4,  8, -4, -4, -4, -4],
       [-8,  8,  4,  4,  0,  4,  4,  4],
       [ 8, -4,  4,  4,  0,  4,  0,  4],
       [ 8, -4,  4,  8,  0,  0,  0, -4],
       [ 8, -4,  4,  8,  0,  0, -4, -8],
       [ 8, -4,  8,  8,  0,  0,  0,  0]])

array([[ 14,  28,   6,   4,   4,   6,   8,   6],
       [ 22,  24,   8,  10,  -2,  -2,  -4,  -2],
       [-10,   8,   6,   8,  -4,  -4,  -6,  -4],
       [ -8,  10,   6,   4,   2,   4,   6,   4],
       [ 10,  -4,   4,   6,   2,   4,   2,   6],
       [  8,  -6,   6,   8,   0,   2,  -2,  -4],
       [  8,  -6,   4,  10,   2,  -2,  -6,  -8],
       [  8,  -4,   8,   8,   2,   2,   0,  -2]])

array([[ 14,  28,   7,   4,   4,   6,   8,   6],
       [ 22,  24,   9,  10,  -2,  -3,  -4,  -2],
       [-10,   8,   6,   8,  -4,  -5,  -6,  -5],
       [ -9,  11,   7,   4,   3,   4,   6,   5],
       [ 10,  -5,   4,   7,   2,   4,   2,   6],
       [  9,  -7,   6,   8,   1,   3,  -2,  -4],
       [  9,  -6,   5,  10,   3,  -2,  -6,  -8],
       [  8,  -5,   8,   9,   2,   3,  -1,  -2]])

MSE: [42.297 15.797  3.047  0.359  0.   ]


К преобразованию с threshold = 4 добавляем половину порогового уровня (с учетом знака).

In [7]:
ht_st_arr = f.copy()
for idx in indx_order:
    ht_st_arr[idx] = st_arr[2][idx] + np.sign(st_arr[2][idx])*threshold[2]//2
display(ht_st_arr)
print('Threshold = 4 MSE:', np.round(np.square(f - st_arr[2]).mean(),3))
print('Half-thresh.  MSE:', np.round(np.square(f - ht_st_arr).mean(),3))

array([[ 14,  30,   6,   6,   6,   6,  10,   6],
       [ 22,  26,  10,  10,   0,   0,  -6,   0],
       [-10,  10,   6,  10,  -6,  -6,  -6,  -6],
       [-10,  10,   6,   6,   0,   6,   6,   6],
       [ 10,  -6,   6,   6,   0,   6,   0,   6],
       [ 10,  -6,   6,  10,   0,   0,   0,  -6],
       [ 10,  -6,   6,  10,   0,   0,  -6, -10],
       [ 10,  -6,  10,  10,   0,   0,   0,   0]])

Threshold = 4 MSE: 3.047
Half-thresh.  MSE: 2.609


Remember to transmit:
- initial threshold value ${T_0}$,
- information on the dimensions of the image,
- how many levels of transform were computed.

In [8]:
encoded_img = {
    'initial threshold': 32,
    'dimensions': 8*8,
    'transform lvl': 2}

In [9]:
pos = 0
pos_prev = 0
encoded_line = str()
for idx in indx_order:
    if (abs(st_arr[2][idx]) >= threshold[1]):
        if (abs(st_arr[2][idx]) < threshold[0]):
            encoded_line += encode(np.sign(st_arr[2][idx])*(scan_order[idx] - pos))
        else:
            pos_prev += 1
        pos = scan_order[idx].copy()
        #print(st_arr[2][idx])
        #print(scan_order[idx] - pos)
        #print(encoded_line)
        
# The number of steps to the last position in the scan order plus 1.
encoded_line += encode(scan_order[indx_order[-1]] + 1 - pos)
# A plus sign is attached to mark the end of the bits for the last binary expansion.
encoded_line += '+'
print(encoded_line)

# At the end of the refinement pass we would have encoded the bits indicating whether refinements are needed for the old significant values.
print('The number of old significant bits:', pos_prev)

+++-1-+++00+110+1++++01+01++-011+10+
The number of old significant bits: 3


In [10]:
import json

# Write the data to a JSON file
with open("encoded_img.json", "w") as json_file:
    json.dump(encoded_img, json_file, indent=4)

print("JSON file created successfully.")

JSON file created successfully.


In [11]:
with open("encoded_img.json", "r") as json_file:
    data = json.load(json_file)
print(data)

{'initial threshold': 32, 'dimensions': 64, 'transform lvl': 2}


In [12]:
Nrow = 8
Ncol = 10

g = np.random.randint(low=-10, high=11, size=Nrow*Ncol).reshape(Nrow,Ncol)

In [13]:
from Functions.scanorder import zigzag_order
from Functions.scanorder import horizont_order

In [14]:
print(zigzag_order(8, 7, LH_to_RL=True))

[[35 36 46 47 53 54 56]
 [22 34 37 45 48 52 55]
 [21 23 33 38 44 49 51]
 [11 20 24 32 39 43 50]
 [10 12 19 25 31 40 42]
 [ 4  9 13 18 26 30 41]
 [ 3  5  8 14 17 27 29]
 [ 1  2  6  7 15 16 28]]


In [15]:
print(horizont_order(8, 7, L_to_R=True))

ValueError: Roman: Index exceeds number of rows